In [1]:
import random
import h5py
import re
import os
import pandas as pd
import numpy as np

from glob import glob
from copy import deepcopy
clinical_data_path = '/Users/maoss2/PycharmProjects/BRCA_experiments_and_paper/datasets/datasets_repository/nationwidechildrens.org_clinical_patient_brca.csv'
new_clinical_data_path = '/Users/maoss2/PycharmProjects/BRCA_experiments_and_paper/datasets/datasets_repository/new_clinical_brca_file.tsv'
train_diablo_subtype_view = '/Users/maoss2/Documents/Doctorat/DATASET_CANCERS/BRCA_Diablo_Datasets/train_subtype_view.csv'
test_diablo_subtype_view = '/Users/maoss2/Documents/Doctorat/DATASET_CANCERS/BRCA_Diablo_Datasets/test_subtype_view.csv'
mrna_train_view = '/Users/maoss2/Documents/Doctorat/DATASET_CANCERS/BRCA_Diablo_Datasets/train_mrna_view.csv'
mrna_test_view = '/Users/maoss2/Documents/Doctorat/DATASET_CANCERS/BRCA_Diablo_Datasets/test_mrna_view.csv'

colomns_to_be_drop = ['tumor_', 'form_completion_date', 'prospective_collection', 'retrospective_collection', 'birth_days_to',
                     'ethnicity', 'last_contact_days_to', 'death_days_to', 'radiation_treatment_adjuvant', 'pharmaceutical_tx_adjuvant',
                     'initial_pathologic_dx_year', 'method_initial_path_dx', 'method_initial_path_dx_other', 'surgical_procedure_first', 
                     'surgical_procedure_first', 'first_surgical_procedure_other', 'surgery_for_Positive_margins', 'surgery_for_Positive_margins_other', 
                     'margin_status_reexcision', 'axillary_staging_method', 'micromet_detection_by_ihc', 'lymph_nodes_examined_count', 
                     'lymph_nodes_examined', 'lymph_nodes_examined_he_count', 'lymph_nodes_examined_ihc_count', 'metastasis_site_other', 
                      'er_positivity_scale_used', 'er_positivity_scale_other', 'er_positivity_method', 'pr_positivity_scale_used', 
                     'pr_positivity_ihc_intensity_score', 'pr_positivity_scale_other', 'pr_positivity_define_method', 'her2_ihc_score',
                     'her2_positivity_scale_other', 'her2_positivity_method_text', 'her2_and_cent17_scale_other', 'her2_fish_method', 
                     'nte_er_status_ihc__Positive', 'nte_er_ihc_intensity_score', 'nte_er_positivity_other_scale', 
                      'nte_er_positivity_define_method', 'nte_pr_status_ihc__Positive', 'nte_pr_ihc_intensity_score', 'nte_pr_positivity_define_method',
                     'nte_her2_status_ihc__Positive', 'nte_her2_positivity_ihc_score', 'nte_her2_positivity_other_scale', 'nte_her2_positivity_method', 'nte_her2_fish_status', 
                      'nte_her2_signal_number', 'nte_cent_17_signal_number', 'her2_cent17_counted_cells_count', 'nte_cent_17_her2_ratio', 'nte_cent17_her2_other_scale',
                      'nte_her2_fish_define_method', 'clinical_M', 'clinical_N', 'clinical_T', 'clinical_stage', 
                     'days_to_first_complete_response', 'days_to_first_partial_response', 'days_to_first_response', 'days_to_initial_pathologic_diagnosis', 
                      'days_to_patient_progression_free', 'days_to_tumor_progression', 'disease_code', 'er_disease_extent_prior_er_treatment', 'er_estimated_duration_response',
                      'er_response_type', 'er_solid_tumor_response_documented_type', 'er_solid_tumor_response_documented_type_other', 'extranodal_involvement',
                      'field', 'histological_type', 'history_of_radiation_metastatic_site', 'history_of_radiation_primary_site', 'history_prior_surgery_indicator',
                      'history_prior_surgery_type', 'history_prior_surgery_type_other', 'icd_10', 'icd_o_3_site', 'informed_consent_verified', 'metastatic_tumor_indicator',
                      'molecular_abnormality_results', 'molecular_abnormality_results_other', 'project_code', 'site_of_primary_tumor_other', 
                     'stage_other', 'tumor_tissue_site', 'axillary_staging_method_other', 'anatomic_neoplasm_subdivision', 
                     'icd_o_3_histology', 'patient_progression_status', 'nte_pr_positivity_other_scale', 'metastasis_site'] 

In [2]:
clinical = pd.read_csv(clinical_data_path)
clinical.drop(clinical.index[0:2], inplace=True)
clinical.replace(to_replace=['[Not Evaluated]', '[Not Available]'], value=['Absent', 'Absent'],
                     inplace=True)
clinical.fillna(value='Absent', inplace=True)
clinical_copy_numpy = deepcopy(clinical.to_numpy())
nb_columns = 129

In [3]:
new_clinical = pd.read_csv(new_clinical_data_path, sep='\t', encoding='ISO-8859-1') # The encoding is important here
new_clinical.drop(new_clinical.index[0:2], inplace=True) 
new_clinical.replace(to_replace=['[Not Evaluated]', '[Not Available]', 'not amplified'], value=['Absent', 'Absent', 'Absent'],
                     inplace=True)
new_clinical.fillna(value='Absent', inplace=True)
new_clinical_copy_numpy = deepcopy(new_clinical.to_numpy())
new_clinical.drop(colomns_to_be_drop, axis=1, inplace=True)

In [4]:
# Absent= -1
new_clinical.replace(to_replace=['Absent'], value=[-1], inplace=True)
new_clinical = new_clinical.fillna(-1)
# CLose= -1
new_clinical.replace(to_replace=['Close'], value=[-1], inplace=True)
# All column with YES/NO
new_clinical.replace(to_replace=['Yes','YES', 'No', 'NO', ], value=[1, 1, 0, 0], inplace=True)
# Positive, Negative, Indeterminate
# Positive =1, negative= 2, Indeterminate= 2, Equivocal= 2
# If indeterminate or equivocal treat them as Negative
new_clinical.replace(to_replace=['Positive', 'Negative', 'Indeterminate', 'Equivocal'], 
                     value=[1, 2, 2, 2], inplace=True)
# Gender
new_clinical.replace(to_replace=['FEMALE', 'MALE'], value=[0, 1], inplace=True)
# Menopause Status
new_clinical.replace(to_replace=['Indeterminate (neither Pre or Postmenopausal)',
 'Peri (6-12 months since last menstrual period)', 
 'Post (prior bilateral ovariectomy OR >12 mo since LMP with no prior hysterectomy)', 
 'Pre (<6 months since LMP AND no prior bilateral ovariectomy AND not on estrogen replacement)'], 
                    value=[3, 1, 2, 0], inplace=True)
# Race:
new_clinical.replace(to_replace=['AMERICAN INDIAN OR ALASKA NATIVE', 'ASIAN', 'BLACK OR AFRICAN AMERICAN', 'WHITE'],
                    value=[1, 2, 3, 4], inplace=True)
# Vital Status
new_clinical.replace(to_replace=['TUMOR FREE', 'WITH TUMOR'], value=[0, 1], inplace=True)

new_clinical.replace(to_replace=['Alive', 'Dead'], value=[1, 0], inplace=True)

# histologic_diagnosis_other
new_clinical['histologic_diagnosis_other'] = (new_clinical['histologic_diagnosis_other'] != -1).astype(int)

# ajcc_staging_edition
new_clinical.replace(to_replace=['3rd', '4th', '5th', '6th', '7th'], value=[3, 4, 5, 6, 7], inplace=True)

# ajcc_tumor_pathologic_pt
new_clinical.replace(to_replace=['T1', 'T1a', 'T1b', 'T1c', 'T2', 'T2a', 'T2b', 'T3', 'T3a', 'T4', 'T4b', 'T4d', 'TX'], 
                     value=[1, 1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 4, 5], inplace=True)

# ajcc_nodes_pathologic_pn
new_clinical.replace(to_replace=['N0', 'N0 (i+)', 'N0 (i-)', 'N0 (mol+)', 'N1', 'N1a', 'N1b', 'N1c', 'N1mi', 'N2', 
                                 'N2a', 'N3', 'N3a', 'N3b', 'N3c', 'NX'], 
                     value=[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 4], inplace=True)

# ajcc_metastasis_pathologic_pm
new_clinical.replace(to_replace=['M0', 'M1', 'MX', 'cM0 (i+)'], value=[0, 1, 2, 3], inplace=True)

# ajcc_pathologic_tumor_stage
new_clinical.replace(to_replace=['Stage I', 'Stage IA', 'Stage IB', 'Stage II', 'Stage IIA', 'Stage IIB', 'Stage III',
                                 'Stage IIIA', 'Stage IIIB', 'Stage IIIC', 'Stage IV', 'Stage X'], 
                     value=[1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 5], inplace=True)

new_clinical.replace(to_replace=['not amplified', 'IHC', '¾Strong'], value=[0, -1, -1], inplace=True)

# In order to build the probability idea:
new_clinical.replace(to_replace=['<10%', '10-19%', '20-29%', '30-39%', '40-49%', '50-59%', '60-69%', '70-79%', '>75%', '80-89%', '90-99%'],
                     value=[1, 1, 1, 2, 2, 3, 3, 4, 4, 4, 4], inplace=True)

new_clinical.replace(to_replace=['>6.0', '>6','<4.0'], value=[6, 6, 4], inplace=True)

# Type Transformation:
new_clinical['age_at_diagnosis'] = new_clinical['age_at_diagnosis'].astype(int)
new_clinical['er_ihc_score'] = new_clinical['er_ihc_score'].astype(int)
new_clinical['pr_status_by_ihc'] = new_clinical['pr_status_by_ihc'].astype(int)
new_clinical['pr_status_ihc_percent_Positive'] = new_clinical['pr_status_ihc_percent_Positive'].astype(int)
new_clinical['her2_copy_number'] = new_clinical['her2_copy_number'].astype(float)
new_clinical['cent17_copy_number'] = new_clinical['cent17_copy_number'].astype(float)
new_clinical['her2_and_cent17_cells_count'] = new_clinical['her2_and_cent17_cells_count'].astype(float)
new_clinical['her2_cent17_ratio'] = new_clinical['her2_cent17_ratio'].astype(float)

# for col in new_clinical.columns:
#     print('column name', col)
#     print(np.unique(new_clinical[col]))

In [5]:
# Build Phenotypes columns
print(new_clinical[['er_status_by_ihc', 'pr_status_by_ihc','her2_status_by_ihc', 'her2_fish_status']].head())
er_labels = new_clinical['er_status_by_ihc']
pr_labels = new_clinical['pr_status_by_ihc']
her_labels = new_clinical['her2_status_by_ihc']
her_labels_copy = deepcopy(her_labels)
her_labels_fish_col = new_clinical['her2_fish_status']

luminal_A = new_clinical.loc[((er_labels == 1) | (pr_labels == 1)) & 
                             ((her_labels == 2) | (her_labels_fish_col == 2))]
luminal_B = new_clinical.loc[((er_labels == 1) | (pr_labels == 1)) & 
                             ((her_labels == 1) | (her_labels_fish_col == 1))]
Her2OverExpression = new_clinical.loc[(er_labels == 2) & (pr_labels == 2) &
                                      ((her_labels == 1) | (her_labels_fish_col == 1))]
TripleNeg = new_clinical.loc[(er_labels == 2) & (pr_labels == 2) &
                         ((her_labels == 2)| (her_labels_fish_col == 2))]

print(luminal_A.shape)
print(luminal_B.shape)
print(Her2OverExpression.shape)
print(TripleNeg.shape)

luminal_A_patients_ids = luminal_A['bcr_patient_barcode'].values
luminal_B_patients_ids = luminal_B['bcr_patient_barcode'].values
Her2OverExpression_patients_ids = Her2OverExpression['bcr_patient_barcode'].values
TripleNeg_patients_ids = TripleNeg['bcr_patient_barcode'].values

phenotype = [None] * (new_clinical.shape[0] + 2)

for idx in new_clinical[new_clinical['bcr_patient_barcode'].isin(luminal_A_patients_ids)].index.to_numpy():
    phenotype[idx] = 'LA'
for idx in new_clinical[new_clinical['bcr_patient_barcode'].isin(luminal_B_patients_ids)].index.to_numpy():
    phenotype[idx] = 'LB'
for idx in new_clinical[new_clinical['bcr_patient_barcode'].isin(Her2OverExpression_patients_ids)].index.to_numpy():
    phenotype[idx] = 'HER2++'
for idx in new_clinical[new_clinical['bcr_patient_barcode'].isin(TripleNeg_patients_ids)].index.to_numpy():
    phenotype[idx] = 'TN'

phenotype = phenotype[2:]
print(len(phenotype))
phenotype_binaire_TN = [-1 if el != 'TN' else 1 for el in phenotype]
print(len(phenotype_binaire_TN))

   er_status_by_ihc  pr_status_by_ihc  her2_status_by_ihc  her2_fish_status
2                 1                 1                   2                -1
3                 1                 1                   1                -1
4                 1                 1                   2                -1
5                 1                 1                   1                -1
6                 1                 1                   2                -1
(653, 34)
(155, 34)
(42, 34)
(170, 34)
1098
1098


In [6]:
# Build probability column
# print(new_clinical[['er_status_ihc_Percent_Positive', 'pr_status_ihc_percent_positive','her2_ihc_percent_positive']].head())
# er_prob = new_clinical['er_status_ihc_Percent_Positive']
# pr_prob = new_clinical['pr_status_ihc_percent_positive']
# her_prob = new_clinical['her2_ihc_percent_positive']
# prob_complet = er_prob + pr_prob + her_prob

# proba = [None] * (new_clinical.shape[0] + 2)

# for idx, el in enumerate(prob_complet):
#     if el >= 5:
#         proba[idx] = 1
#     else:
#         proba[idx] = 0
# proba = proba[:-2]

#TODO: I change the idea to build this colomn: I should have the column available for each patients when i will analyze the results aka build a function that pull everything out

In [7]:
new_clinical['phenotype'] = phenotype
new_clinical['phenotype_TN'] = phenotype_binaire_TN

# Add the label of diablo to the mix

In [8]:
    train_diablo_type = pd.read_csv(train_diablo_subtype_view)
    test_diablo_type = pd.read_csv(test_diablo_subtype_view)
    mrna_train_data = pd.read_csv(mrna_train_view)
    mrna_test_data = pd.read_csv(mrna_test_view)
    train_labels = list(zip(mrna_train_data['Unnamed: 0'].values, train_diablo_type['x'].values))
    test_labels = list(zip(mrna_test_data['Unnamed: 0'].values, test_diablo_type['x'].values))
    diablo_labels = train_labels + test_labels

    luminal_A_mine = new_clinical.loc[new_clinical['phenotype'] == 'LA']
    luminal_A_mine = [el[-4:] for el in luminal_A_mine['bcr_patient_barcode']]
    luminal_A_diablo = [el[0] for el in diablo_labels if el[1] == 'LumA']
    lum_A_commun = [el for el in luminal_A_diablo if el in luminal_A_mine]
    lum_A_diff = [el for el in luminal_A_diablo if el not in luminal_A_mine]
    lum_A_diff_mine_not_in_diablo = [el for el in luminal_A_mine if el not in luminal_A_diablo]

    luminal_B_mine = new_clinical.loc[new_clinical['phenotype'] == 'LB']
    luminal_B_mine = [el[-4:] for el in luminal_B_mine['bcr_patient_barcode']]
    luminal_B_diablo = [el[0] for el in diablo_labels if el[1] == 'LumB']
    lum_B_commun = [el for el in luminal_B_diablo if el in luminal_B_mine]
    lum_B_diff = [el for el in luminal_B_diablo if el not in luminal_B_mine]
    lum_B_diff_mine_not_in_diablo = [el for el in luminal_B_mine if el not in luminal_B_diablo]

    her2_mine  = new_clinical.loc[new_clinical['phenotype'] == 'HER2++']
    her2_diablo = [el[0] for el in diablo_labels if el[1] == 'Her2']
    her2_commun = [el for el in her2_diablo if el in her2_mine]
    her2_diff = [el for el in her2_diablo if el not in her2_mine]
    her2_diff_mine_not_in_diablo = [el for el in her2_mine if el not in her2_diablo]

    TripleNeg_basal_mine = new_clinical.loc[new_clinical['phenotype'] == 'TN']
    TripleNeg_basal_mine = [el[-4:] for el in TripleNeg_basal_mine['bcr_patient_barcode']]
    basal_diablo = [el[0] for el in diablo_labels if el[1] == 'Basal']
    TripleNeg_basal_commun = [el for el in basal_diablo if el in TripleNeg_basal_mine]
    TripleNeg_basal_diff = [el for el in basal_diablo if el not in TripleNeg_basal_mine]
    TripleNeg_basal_diff_mine_not_in_diablo = [el for el in TripleNeg_basal_mine if el not in basal_diablo]

    tcga_patients_names = new_clinical['bcr_patient_barcode'].values[2:]
    tcga_patients_names = [(el, el[-4:]) for el in tcga_patients_names]
    non_basal_diablo = []
    non_basal_diablo.extend(luminal_A_diablo)
    non_basal_diablo.extend(luminal_B_diablo)
    non_basal_diablo.extend(her2_diablo)
    
    luminal_A_diablo_real_name = [name[0] for name in tcga_patients_names if name[1] in luminal_A_diablo]
    luminal_B_diablo_real_name = [name[0] for name in tcga_patients_names if name[1] in luminal_B_diablo]
    her2_diablo_real_name = [name[0] for name in tcga_patients_names if name[1] in her2_diablo]
    basal_diablo_real_name = [name[0] for name in tcga_patients_names if name[1] in basal_diablo]
    

    diablo_NON_TN = luminal_A_diablo_real_name + luminal_B_diablo_real_name + her2_diablo_real_name

    diablo_phenotype = [None] * (new_clinical.shape[0] + 2)
    diablo_phenotype_TN_AND_BASAL = [None] * (new_clinical.shape[0] + 2)

    for idx in new_clinical[new_clinical['bcr_patient_barcode'].isin(diablo_NON_TN)].index.to_numpy():
        diablo_phenotype[idx] = -1
        diablo_phenotype_TN_AND_BASAL[idx] = 'NON TN'
    for name in basal_diablo_real_name:
        if name in TripleNeg_basal_commun:
            idx = new_clinical[new_clinical['bcr_patient_barcode'] == name].index.to_numpy()[0]
            diablo_phenotype[idx] = 1
            diablo_phenotype_TN_AND_BASAL[idx] = 'TN'
        else:
            idx = new_clinical[new_clinical['bcr_patient_barcode'] == name].index.to_numpy()[0]
            diablo_phenotype[idx] = 1
            diablo_phenotype_TN_AND_BASAL[idx] = 'BASAL NON TN'
    diablo_phenotype = diablo_phenotype[2:]
    diablo_phenotype_TN_AND_BASAL = diablo_phenotype_TN_AND_BASAL[2:]

In [9]:
new_clinical['phenotype_DIABLO'] = diablo_phenotype
new_clinical['phenotype_DIABLO_TN_AND_BASAL'] = diablo_phenotype_TN_AND_BASAL

In [10]:
new_clinical[['er_status_by_ihc', 'pr_status_by_ihc','her2_status_by_ihc', 'her2_fish_status', 'er_status_ihc_Percent_Positive', 'pr_status_ihc_percent_Positive','her2_ihc_percent_Positive', 'phenotype']].head()

,er_status_by_ihc,pr_status_by_ihc,her2_status_by_ihc,her2_fish_status,er_status_ihc_Percent_Positive,pr_status_ihc_percent_Positive,her2_ihc_percent_Positive,phenotype
2,1,1,2,-1,3,3,-1,LA
3,1,1,1,-1,1,1,-1,LB
4,1,1,2,-1,4,2,-1,LA
5,1,1,1,-1,4,4,-1,LB
6,1,1,2,-1,3,4,1,LA


 # Save Labels phenotype and binary phenotypes with probability scores

In [11]:
new_clinical[['bcr_patient_barcode', 'phenotype', 'phenotype_TN', 'phenotype_DIABLO', 'phenotype_DIABLO_TN_AND_BASAL', 'er_status_ihc_Percent_Positive', 'pr_status_ihc_percent_Positive', 'her2_ihc_percent_Positive']].to_csv('clinical_views_labels_and_proba.tsv', sep='\t')

# Save CLinical FIle to be used for the clinical view 

In [12]:
new_clinical.drop(['er_status_by_ihc', 'er_status_ihc_Percent_Positive', 'er_ihc_score',
       'pr_status_by_ihc', 'pr_status_ihc_percent_Positive',
       'her2_status_by_ihc', 'her2_ihc_percent_Positive', 'her2_fish_status','nte_er_status',
       'nte_pr_status_by_ihc', 'nte_her2_status', 'patient_id',
       'tissue_source_site', 'phenotype', 'phenotype_TN', 'phenotype_DIABLO', 'phenotype_DIABLO_TN_AND_BASAL','her2_cent17_ratio', 'her2_and_cent17_cells_count'], axis=1, inplace=True)
new_clinical.to_csv('new_clinical_view.tsv', sep='\t')

In [13]:
labels_file = 'clinical_views_labels_and_proba.tsv'

In [14]:
labels = pd.read_csv('{}'.format(labels_file),  index_col="bcr_patient_barcode", sep="\t")
# labels = pd.read_csv('{}'.format(labels_file), index_col="bcr_patient_barcode", sep="\t")
index_pos = labels.loc[labels['phenotype_TN'] == 1].index.values
index_neg = labels.loc[labels['phenotype_TN'] == -1].index.values

In [15]:
labels.head()

,Unnamed: 0,phenotype,phenotype_TN,phenotype_DIABLO,phenotype_DIABLO_TN_AND_BASAL,er_status_ihc_Percent_Positive,pr_status_ihc_percent_Positive,her2_ihc_percent_Positive
bcr_patient_barcode,,,,,,,,
TCGA-3C-AAAU,2,LA,-1,NaN,NaN,3,3,-1
TCGA-3C-AALI,3,LB,-1,NaN,NaN,1,1,-1
TCGA-3C-AALJ,4,LA,-1,NaN,NaN,4,2,-1
TCGA-3C-AALK,5,LB,-1,NaN,NaN,4,4,-1
TCGA-4H-AAAK,6,LA,-1,NaN,NaN,3,4,1


# Build final label to go to the learning phase

In [16]:
d = pd.read_csv('clinical_views_labels_and_proba.tsv', sep='\t')
d.fillna(1000, inplace=True)

In [17]:
d.columns

Index(['Unnamed: 0', 'bcr_patient_barcode', 'phenotype', 'phenotype_TN',
       'phenotype_DIABLO', 'phenotype_DIABLO_TN_AND_BASAL',
       'er_status_ihc_Percent_Positive', 'pr_status_ihc_percent_Positive',
       'her2_ihc_percent_Positive'],
      dtype='object')

In [18]:
data_phenotype_TN_different_celui_de_DIABLO = d[['bcr_patient_barcode','phenotype_TN', 'phenotype_DIABLO']][(d['phenotype_TN'] != d['phenotype_DIABLO']) & (d['phenotype_DIABLO'] != 1000.0)]
# Size= 63 patients
print(data_phenotype_TN_different_celui_de_DIABLO.shape)
print(data_phenotype_TN_different_celui_de_DIABLO[data_phenotype_TN_different_celui_de_DIABLO['phenotype_TN'] != -1].shape)
print(data_phenotype_TN_different_celui_de_DIABLO.head())
liste_patients_phenotype_TN_different_celui_de_DIABLO = data_phenotype_TN_different_celui_de_DIABLO['bcr_patient_barcode'].values

(63, 3)
(19, 3)
    bcr_patient_barcode  phenotype_TN  phenotype_DIABLO
56         TCGA-A2-A0EQ             1              -1.0
105        TCGA-A2-A25F            -1               1.0
116        TCGA-A2-A3Y0            -1               1.0
118        TCGA-A2-A4RX            -1               1.0
121        TCGA-A2-A4S1            -1               1.0


# Analyses Personelles 1
19 patients sont labelisés TN chez moi mais NON TN chez Diablo/ 
Du coup: 44 sont BASAL non TN dans Diablo versus mine
On va simplement les enlevés du dataset: comme ca on evite déjà l'ambiguité

In [19]:
d[d['phenotype'] == 1000.0]

,Unnamed: 0,bcr_patient_barcode,phenotype,phenotype_TN,phenotype_DIABLO,phenotype_DIABLO_TN_AND_BASAL,er_status_ihc_Percent_Positive,pr_status_ihc_percent_Positive,her2_ihc_percent_Positive
127,129,TCGA-A7-A0CH,1000,-1,-1.0,NON TN,-1,-1,-1
251,253,TCGA-AC-A23E,1000,-1,-1.0,NON TN,2,4,-1
258,260,TCGA-AC-A2FE,1000,-1,-1.0,NON TN,3,1,-1
262,264,TCGA-AC-A2FM,1000,-1,-1.0,NON TN,-1,-1,-1
279,281,TCGA-AC-A3YJ,1000,-1,-1.0,NON TN,-1,-1,-1
...,...,...,...,...,...,...,...,...,...
1074,1076,TCGA-PL-A8LV,1000,-1,1000.0,1000,-1,-1,-1
1075,1077,TCGA-PL-A8LX,1000,-1,1000.0,1000,-1,-1,-1
1076,1078,TCGA-PL-A8LY,1000,-1,1000.0,1000,-1,-1,-1
1077,1079,TCGA-PL-A8LZ,1000,-1,1.0,BASAL NON TN,-1,-1,-1


In [20]:
liste_patients_phenotype_inconnus = d[d['phenotype'] == 1000.0]['bcr_patient_barcode'].values

# Analyses Personelles 2
Size: 117 patients: DOnc on a 117 patients que l'on a pas pu définir clairement en type du genre: LA/LB/TN or Her2+. 
Les informations related to the stage and the TNM notation ne permettent pas de dire c'est quoi le type. 
Mais on peut partir du fait que ici c'est forcément label -1 car c'est clair (sur???) que c'est pas des TN!
On va la jouer safe since our results are not good so far: On va les eliminer du datasets également.  
Toutes ces modifications seront faites et sauvegardées dans le meme fichier mais "_copie.tsv"
Va faloir reouvrir l'autre fichier clinical? Et supprimer too? Ou 
Conclusion
-------
1. Eliminer les patients dont nous meme on a pas pu définir les labels (a partir des 3 recepteurs). 117
2. Eliminer le spatients pour lequels on a de l’ambiguïté au niveau de la labelisation entre celle que j'ai et celle de Diablo: 63
3. Il devrait y avoir un merge entre les 117 et 63 pour eliminer les patients
4. Reconstruire les fichiers externe avec la terminaison "_copie.tsv" pour savoir ce qu'on manipule

In [21]:
o = list(liste_patients_phenotype_TN_different_celui_de_DIABLO)
l = list(liste_patients_phenotype_inconnus)
o.extend(l)
liste_patients_to_be_deleted = np.unique(o)

In [22]:
# d.drop(d.loc[d['bcr_patient_barcode'].isin(liste_patients_to_be_deleted)])
temp = d.loc[~d['bcr_patient_barcode'].isin(liste_patients_to_be_deleted)]
temp.drop(['Unnamed: 0'], axis=1, inplace=True)
temp.to_csv('clinical_views_labels_and_proba_copie.csv')

/Users/maoss2/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [24]:
d = pd.read_csv('new_clinical_view.tsv', sep='\t')
temp = d.loc[~d['bcr_patient_barcode'].isin(liste_patients_to_be_deleted)]
temp.drop(['Unnamed: 0'], axis=1, inplace=True)
temp.to_csv('new_clinical_view_copie.tsv', sep='\t')